# SIRENE API Client - Data Models and Extraction Demo

This notebook demonstrates the core data models and extraction capabilities of the SIRENE API client, based on the Django Celery example but focused purely on the API client functionality.

## What You'll Learn

1. **Data Models**: Understanding the structure of Company, Facility, Address, and other data models
2. **Extraction Methods**: Different ways to extract SIREN data (company only vs full extraction)
3. **Progress Tracking**: How to monitor extraction progress
4. **Error Handling**: Dealing with API errors and validation issues
5. **Data Relationships**: How different entities relate to each other

## Setup

First, let's import the necessary modules and set up our environment.


# SIRENE API Client - Data Models and Extraction Demo

This notebook demonstrates the core data models and extraction capabilities of the SIRENE API client, based on the Django Celery example but focused purely on the API client functionality.

## What You'll Learn

1. **Data Models**: Understanding the structure of Company, Facility, Address, and other data models
2. **Extraction Methods**: Different ways to extract SIREN data (company only vs full extraction)
3. **Progress Tracking**: How to monitor extraction progress
4. **Error Handling**: Dealing with API errors and validation issues
5. **Data Relationships**: How different entities relate to each other

## Setup

First, let's import the necessary modules and set up our environment.


In [1]:
# Configuration
# Note: You'll need to set your SIRENE API token as an environment variable
# export SIRENE_API_TOKEN="your_token_here"

import os
from datetime import datetime
from sirene_api_client import (
    AuthenticatedClient,
    ETLConfig,
    ValidationMode,
)

API_BASE_URL = "https://api.insee.fr/api-sirene/3.11"
API_TOKEN = os.getenv("SIRENE_API_TOKEN")

if not API_TOKEN:
    print("⚠️  Warning: SIRENE_API_TOKEN environment variable not set")
    print("   Please set it with: export SIRENE_API_TOKEN='your_token_here'")
    print(
        "   You can get a token from: https://api.insee.fr/catalogue/site/themes/wso2/subthemes/insee/pages/item-info.jag?name=Sirene&version=V3&provider=insee"
    )
else:
    print(f"✅ API Token configured (length: {len(API_TOKEN)} characters)")

# Initialize client and config
# IMPORTANT: Set prefix="" to avoid adding "Bearer " to the token
client = AuthenticatedClient(
    base_url=API_BASE_URL,
    token=API_TOKEN,
    prefix="",  # No prefix for INSEE API
    auth_header_name="X-INSEE-Api-Key-Integration",
)
config = ETLConfig(validation_mode=ValidationMode.STRICT)

print(f"🔧 Client configured with base URL: {API_BASE_URL}")
print(f"🔧 Validation mode: {config.validation_mode}")

✅ API Token configured (length: 36 characters)
🔧 Client configured with base URL: https://api.insee.fr/api-sirene/3.11
🔧 Validation mode: ValidationMode.STRICT


In [2]:
# Test client connection
from sirene_api_client.api.informations import informations


def test_client_connection():
    if API_TOKEN:
        try:
            print("🔍 Testing client connection...")
            print(f"   API URL: {API_BASE_URL}")
            print(f"   Token: {API_TOKEN[:8]}...")

            # Test with a simple API call (get service information)
            response = informations.sync(client=client)

            print("✅ Client connection successful!")
            print(f"   Response type: {type(response)}")

            if response:
                print(f"   Service status: {response.etat_service}")
                print(f"   Version: {response.version_service}")
                print(f"   Header: {response.header}")
                return True
            else:
                print("   ❌ No response data")
                return False

        except Exception as e:
            print(f"❌ Client connection failed: {e}")
            print(f"   Error type: {type(e).__name__}")
            return False
    else:
        print("⏭️  No API token - cannot test connection")
        return False


# Run the test
connection_ok = test_client_connection()

🔍 Testing client connection...
   API URL: https://api.insee.fr/api-sirene/3.11
   Token: 33cf9797...
✅ Client connection successful!
   Response type: <class 'sirene_api_client.models.reponse_informations.ReponseInformations'>
   Service status: UP
   Version: 3.11.80
   Header: Header(statut=200, message='OK', total=<sirene_api_client.api_types.Unset object at 0x0000023F2634D010>, debut=<sirene_api_client.api_types.Unset object at 0x0000023F2634D010>, nombre=<sirene_api_client.api_types.Unset object at 0x0000023F2634D010>, curseur=<sirene_api_client.api_types.Unset object at 0x0000023F2634D010>, curseur_suivant=<sirene_api_client.api_types.Unset object at 0x0000023F2634D010>, additional_properties={})


## 1. Understanding Data Models

Let's explore the different data models available in the SIRENE API client. These models represent the structured data you can extract from the SIRENE database.

In [3]:
# Let's examine the available data models
from sirene_api_client.etl.models import *

# List all available model classes
model_classes = [
    "CompanyData",
    "FacilityData",
    "AddressData",
    "CompanyIdentifierData",
    "FacilityIdentifierData",
    "CompanyLegalUnitPeriodData",
    "FacilityEstablishmentPeriodData",
    "SIRENExtractResult",
    "ActivityClassificationData",
    "FacilityOwnershipData",
    "ExternalRegistryRecordData",
]

print("📋 Available Data Models:")
for model in model_classes:
    if model in globals():
        print(f"  ✅ {model}")
    else:
        print(f"  ❌ {model} (not found)")

print("\n🔍 Let's examine the structure of key models:")

📋 Available Data Models:
  ✅ CompanyData
  ✅ FacilityData
  ✅ AddressData
  ✅ CompanyIdentifierData
  ✅ FacilityIdentifierData
  ✅ CompanyLegalUnitPeriodData
  ✅ FacilityEstablishmentPeriodData
  ✅ SIRENExtractResult
  ✅ ActivityClassificationData
  ✅ FacilityOwnershipData
  ✅ ExternalRegistryRecordData

🔍 Let's examine the structure of key models:


In [4]:
# Examine model structures using introspection
def examine_model(model_class):
    """Examine a model class and show its fields and methods."""
    print(f"\n📊 {model_class.__name__}:")
    print(f"   Module: {model_class.__module__}")

    # Get all attributes
    attrs = [attr for attr in dir(model_class) if not attr.startswith("_")]

    # Separate fields from methods
    fields = []
    methods = []

    for attr in attrs:
        obj = getattr(model_class, attr)
        if callable(obj):
            methods.append(attr)
        else:
            fields.append(attr)

    if fields:
        print(f"   Fields: {', '.join(fields)}")
    if methods:
        print(
            f"   Methods: {', '.join(methods[:5])}{'...' if len(methods) > 5 else ''}"
        )


# Examine key models
key_models = ["CompanyData", "FacilityData", "AddressData", "SIRENExtractResult"]
for model_name in key_models:
    if model_name in globals():
        examine_model(globals()[model_name])


📊 CompanyData:
   Module: sirene_api_client.etl.models
   Fields: model_computed_fields, model_config, model_extra, model_fields, model_fields_set
   Methods: construct, copy, dict, from_orm, json...

📊 FacilityData:
   Module: sirene_api_client.etl.models
   Fields: model_computed_fields, model_config, model_extra, model_fields, model_fields_set
   Methods: construct, copy, dict, from_orm, json...

📊 AddressData:
   Module: sirene_api_client.etl.models
   Fields: model_computed_fields, model_config, model_extra, model_fields, model_fields_set
   Methods: construct, copy, dict, from_orm, json...

📊 SIRENExtractResult:
   Module: sirene_api_client.etl.models
   Fields: model_computed_fields, model_config, model_extra, model_fields, model_fields_set
   Methods: construct, copy, dict, export_to_json, from_orm...


## 2. Extraction Methods Comparison

Now let's see the different ways to extract SIREN data. We'll use a real SIREN number for demonstration.


In [5]:
# Test SIREN number (a real company for demonstration)
# This is the SIREN for "SOCIETE GENERALE" - a major French bank
TEST_SIREN = "061500542"

print(f"🎯 Using test SIREN: {TEST_SIREN}")
print("   This is SOCIETE NATIONALE SNCF - a major French bank")
print("   (We'll use this for all our examples)")

# Check if we have API access
if not API_TOKEN:
    print("\n⚠️  Skipping API calls - no token configured")
    print("   Set SIRENE_API_TOKEN environment variable to run live examples")
else:
    print(f"\n✅ Ready to make API calls with token")

🎯 Using test SIREN: 061500542
   This is SOCIETE NATIONALE SNCF - a major French bank
   (We'll use this for all our examples)

✅ Ready to make API calls with token


### Method 1: Company-Only Extraction (Fast)

This method extracts only the company information without facilities. It's fast and provides immediate feedback.


In [6]:
# Method 1: Extract company data only
from sirene_api_client.etl import extract_company_only


async def test_company_extraction():
    if API_TOKEN:
        try:
            print("🚀 Extracting company data only...")
            start_time = datetime.now()

            # Extract company data and facility count
            company_data, facility_count = await extract_company_only(
                TEST_SIREN, client, config
            )

            end_time = datetime.now()
            duration = (end_time - start_time).total_seconds()

            print(f"✅ Extraction completed in {duration:.2f} seconds")
            print(f"📊 Found {facility_count} facilities")

            # Display company information
            print(f"\n🏢 Company Information:")
            print(f"   Name: {company_data.name}")
            print(
                f"   SIREN: {company_data.identifiers[0].value if company_data.identifiers else 'N/A'}"
            )
            print(f"   Legal form: {getattr(company_data, 'legal_form', 'N/A')}")
            print(f"   Activity: {getattr(company_data, 'activity', 'N/A')}")

            # Show available attributes
            print(f"\n🔍 Available company attributes:")
            company_attrs = [
                attr for attr in dir(company_data) if not attr.startswith("_")
            ]
            for attr in company_attrs[:10]:  # Show first 10 attributes
                value = getattr(company_data, attr, None)
                if not callable(value):
                    print(f"   {attr}: {type(value).__name__}")

            return company_data, facility_count

        except Exception as e:
            print(f"❌ Error during company extraction: {e}")
            print(f"   Error type: {type(e).__name__}")
            return None, 0
    else:
        print("⏭️  Skipping - no API token")
        return None, 0


# Run the async function
company_data, facility_count = await test_company_extraction()

🚀 Extracting company data only...
✅ Extraction completed in 0.88 seconds
📊 Found 33 facilities

🏢 Company Information:
   Name: BDO RHONE-ALPES
   SIREN: 061500542
   Legal form: N/A
   Activity: N/A

🔍 Available company attributes:
   acronym: str
   association_id: NoneType
   category_year: int
   company_size_category: str
   creation_date: date
   diffusion_status: str
   employee_band: str


### Method 2: Full Extraction with Progress Tracking

This method extracts all data (company + facilities + addresses) with real-time progress updates.


In [7]:
# Method 2: Full extraction with progress tracking
from sirene_api_client.etl import extract_and_transform_siren_with_progress


async def run_full_extraction():
    if API_TOKEN:
        try:
            print("🚀 Starting full extraction with progress tracking...")

            # Progress tracking variables
            progress_updates = []

            def progress_callback(update):
                """Callback function to track progress updates."""
                progress_updates.append(update)
                print(
                    f"📈 Progress: {update.get('phase', 'Unknown')} - "
                    f"{update.get('processed_facilities', 0)}/{update.get('total_facilities', 0)} facilities"
                )

            start_time = datetime.now()

            # Extract all data with progress tracking (note the await!)
            result = await extract_and_transform_siren_with_progress(
                TEST_SIREN, client, config, progress_callback=progress_callback
            )

            end_time = datetime.now()
            duration = (end_time - start_time).total_seconds()

            print(f"\n✅ Full extraction completed in {duration:.2f} seconds")
            print(f"📊 Progress updates received: {len(progress_updates)}")

            # Display summary
            print(f"\n📋 Extraction Summary:")
            print(f"   Company: {result.company.name}")
            print(f"   Facilities: {len(result.facilities)}")
            print(f"   Addresses: {len(result.addresses)}")
            print(f"   Legal periods: {len(result.legal_unit_periods)}")
            print(f"   Establishment periods: {len(result.establishment_periods)}")

            return result

        except Exception as e:
            print(f"❌ Error during full extraction: {e}")
            print(f"   Error type: {type(e).__name__}")
            return None
    else:
        print("⏭️  Skipping - no API token")
        return None


# Run the async function
result = await run_full_extraction()

🚀 Starting full extraction with progress tracking...
📈 Progress: company_extracted - 0/0 facilities
📈 Progress: facilities_processing - 1/33 facilities
📈 Progress: facilities_processing - 2/33 facilities
📈 Progress: facilities_processing - 3/33 facilities
📈 Progress: facilities_processing - 4/33 facilities
📈 Progress: facilities_processing - 5/33 facilities
📈 Progress: facilities_processing - 6/33 facilities
📈 Progress: facilities_processing - 7/33 facilities
📈 Progress: facilities_processing - 8/33 facilities
📈 Progress: facilities_processing - 9/33 facilities
📈 Progress: facilities_processing - 10/33 facilities
📈 Progress: facilities_processing - 11/33 facilities
📈 Progress: facilities_processing - 12/33 facilities
📈 Progress: facilities_processing - 13/33 facilities
📈 Progress: facilities_processing - 14/33 facilities
📈 Progress: facilities_processing - 15/33 facilities
📈 Progress: facilities_processing - 16/33 facilities
📈 Progress: facilities_processing - 17/33 facilities
📈 Progre

## 3. Data Model Deep Dive

Let's examine the detailed structure of each data model and their relationships.


In [8]:
# Manual fix for facility names in the notebook
if API_TOKEN and "result" in locals():
    print("🔧 Manual Facility Name Fix")
    print("=" * 40)

    for i, facility in enumerate(result.facilities[:3]):
        print(f"\n🏭 Facility {i + 1}:")
        print(f"   Original name: '{facility.name}'")

        # Find the corresponding establishment period
        if result.establishment_periods:
            # Get the current period for this facility
            current_periods = [
                p for p in result.establishment_periods if p.start and not p.end
            ]
            if current_periods:
                period = current_periods[0]  # Assuming first current period
                print(f"   Current period activity: {period.activity_code}")
                print(f"   Current period status: {period.status}")

        # Check registry records for enseigne data
        establishment_records = [
            r for r in result.registry_records if r.entity_type == "establishment"
        ]
        if establishment_records and i < len(establishment_records):
            est_record = establishment_records[i]
            payload = est_record.payload

            if "periodesEtablissement" in payload and payload["periodesEtablissement"]:
                periods = payload["periodesEtablissement"]
                current_period = None

                # Find current period (no dateFin)
                for period in periods:
                    if not period.get("dateFin"):
                        current_period = period
                        break

                if current_period:
                    enseigne1 = current_period.get("enseigne1Etablissement")
                    enseigne2 = current_period.get("enseigne2Etablissement")
                    enseigne3 = current_period.get("enseigne3Etablissement")

                    print(f"   Enseigne 1: {enseigne1}")
                    print(f"   Enseigne 2: {enseigne2}")
                    print(f"   Enseigne 3: {enseigne3}")

                    # Suggest corrected name
                    if enseigne1:
                        suggested_name = enseigne1
                    elif enseigne2:
                        suggested_name = enseigne2
                    elif enseigne3:
                        suggested_name = enseigne3
                    else:
                        suggested_name = (
                            f"{result.company.name} - Établissement {facility.nic}"
                        )

                    print(f"   ✅ Suggested name: '{suggested_name}'")

else:
    print("⏭️  No extraction results available")

🔧 Manual Facility Name Fix

🏭 Facility 1:
   Original name: 'Unknown Facility'
   Current period activity: 69.20Z
   Current period status: closed
   Enseigne 1: None
   Enseigne 2: None
   Enseigne 3: None
   ✅ Suggested name: 'BDO RHONE-ALPES - Établissement 00040'

🏭 Facility 2:
   Original name: 'BDO'
   Current period activity: 69.20Z
   Current period status: closed
   Enseigne 1: None
   Enseigne 2: None
   Enseigne 3: None
   ✅ Suggested name: 'BDO RHONE-ALPES - Établissement 00057'

🏭 Facility 3:
   Original name: 'Unknown Facility'
   Current period activity: 69.20Z
   Current period status: closed
   Enseigne 1: None
   Enseigne 2: None
   Enseigne 3: None
   ✅ Suggested name: 'BDO RHONE-ALPES - Établissement 00263'


In [9]:
# Debug registry records to see raw facility data
if API_TOKEN and "result" in locals() and result.registry_records:
    print("🔍 Debugging Registry Records (Raw API Data)")
    print("=" * 60)

    # Look for establishment records specifically
    establishment_records = [
        r for r in result.registry_records if r.entity_type == "establishment"
    ]
    print(f"📊 Found {len(establishment_records)} establishment records")

    if establishment_records:
        # Look at first establishment record
        est_record = establishment_records[0]
        print(f"\n🏭 First Establishment Record:")
        print(f"   External ID: {est_record.external_id}")
        print(f"   Entity Type: {est_record.entity_type}")
        print(f"   Registry Updated: {est_record.registry_updated_at}")
        print(f"   Ingested At: {est_record.ingested_at}")

        # Check the raw payload for facility name fields
        payload = est_record.payload
        print(f"\n📋 Raw Payload Keys:")
        for key in sorted(payload.keys()):
            print(f"   {key}")

        # Look for name-related fields
        name_fields = [
            k
            for k in payload.keys()
            if "nom" in k.lower() or "denomination" in k.lower() or "name" in k.lower()
        ]
        print(f"\n🏷️  Name-related fields found:")
        for field in name_fields:
            value = payload.get(field)
            print(f"   {field}: {value}")

        # Check periods data in payload
        if "periodesEtablissement" in payload:
            periods = payload["periodesEtablissement"]
            print(f"\n📅 Establishment Periods in payload:")
            print(f"   Number of periods: {len(periods) if periods else 0}")
            if periods and len(periods) > 0:
                first_period = periods[0]
                print(f"   First period keys: {list(first_period.keys())}")

                # Look for denomination in first period
                denom_fields = [
                    k for k in first_period.keys() if "denomination" in k.lower()
                ]
                print(f"   Denomination fields in first period:")
                for field in denom_fields:
                    value = first_period.get(field)
                    print(f"     {field}: {value}")
        else:
            print(f"\n📅 No 'periodesEtablissement' found in payload")

        # Check if there's a direct facility name field
        direct_name_fields = [
            "denominationUsuelleEtablissement",
            "nomEtablissement",
            "denominationEtablissement",
        ]
        print(f"\n🏷️  Direct facility name fields:")
        for field in direct_name_fields:
            value = payload.get(field)
            print(f"   {field}: {value}")
    else:
        print("   No establishment records found")

    # Also check legal unit records for company name
    legal_records = [
        r for r in result.registry_records if r.entity_type == "legal_unit"
    ]
    print(f"\n🏢 Found {len(legal_records)} legal unit records")

    if legal_records:
        legal_record = legal_records[0]
        payload = legal_record.payload
        print(f"   Company name fields:")
        name_fields = [
            k
            for k in payload.keys()
            if "denomination" in k.lower() or "nom" in k.lower()
        ]
        for field in name_fields:
            value = payload.get(field)
            print(f"     {field}: {value}")

else:
    print("⏭️  No registry records available for debugging")

🔍 Debugging Registry Records (Raw API Data)
📊 Found 33 establishment records

🏭 First Establishment Record:
   External ID: 06150054200040
   Entity Type: establishment
   Registry Updated: 2024-03-30 10:22:44.251000
   Ingested At: 2025-10-23 14:44:13.724458

📋 Raw Payload Keys:
   adresse2Etablissement
   adresseEtablissement
   dateCreationEtablissement
   dateDernierTraitementEtablissement
   etablissementSiege
   nic
   nombrePeriodesEtablissement
   periodesEtablissement
   siren
   siret
   statutDiffusionEtablissement
   trancheEffectifsEtablissement
   uniteLegale

🏷️  Name-related fields found:
   nombrePeriodesEtablissement: 5

📅 Establishment Periods in payload:
   Number of periods: 5
   First period keys: ['dateDebut', 'etatAdministratifEtablissement', 'changementEtatAdministratifEtablissement', 'changementEnseigneEtablissement', 'changementDenominationUsuelleEtablissement', 'activitePrincipaleEtablissement', 'nomenclatureActivitePrincipaleEtablissement', 'changementActiv

In [10]:
# Debug registry records to see raw facility data
if API_TOKEN and "result" in locals() and result.registry_records:
    print("🔍 Debugging Registry Records (Raw API Data)")
    print("=" * 60)

    # Look for establishment records specifically
    establishment_records = [
        r for r in result.registry_records if r.entity_type == "establishment"
    ]
    print(f"📊 Found {len(establishment_records)} establishment records")

    if establishment_records:
        # Look at first establishment record
        est_record = establishment_records[0]
        print(f"\n🏭 First Establishment Record:")
        print(f"   External ID: {est_record.external_id}")
        print(f"   Entity Type: {est_record.entity_type}")
        print(f"   Registry Updated: {est_record.registry_updated_at}")
        print(f"   Ingested At: {est_record.ingested_at}")

        # Check the raw payload for facility name fields
        payload = est_record.payload
        print(f"\n📋 Raw Payload Keys:")
        for key in sorted(payload.keys()):
            print(f"   {key}")

        # Look for name-related fields
        name_fields = [
            k
            for k in payload.keys()
            if "nom" in k.lower() or "denomination" in k.lower() or "name" in k.lower()
        ]
        print(f"\n🏷️  Name-related fields found:")
        for field in name_fields:
            value = payload.get(field)
            print(f"   {field}: {value}")

        # Check periods data in payload
        if "periodesEtablissement" in payload:
            periods = payload["periodesEtablissement"]
            print(f"\n📅 Establishment Periods in payload:")
            print(f"   Number of periods: {len(periods) if periods else 0}")
            if periods and len(periods) > 0:
                first_period = periods[0]
                print(f"   First period keys: {list(first_period.keys())}")

                # Look for denomination in first period
                denom_fields = [
                    k for k in first_period.keys() if "denomination" in k.lower()
                ]
                print(f"   Denomination fields in first period:")
                for field in denom_fields:
                    value = first_period.get(field)
                    print(f"     {field}: {value}")
        else:
            print(f"\n📅 No 'periodesEtablissement' found in payload")

        # Check if there's a direct facility name field
        direct_name_fields = [
            "denominationUsuelleEtablissement",
            "nomEtablissement",
            "denominationEtablissement",
        ]
        print(f"\n🏷️  Direct facility name fields:")
        for field in direct_name_fields:
            value = payload.get(field)
            print(f"   {field}: {value}")
    else:
        print("   No establishment records found")

    # Also check legal unit records for company name
    legal_records = [
        r for r in result.registry_records if r.entity_type == "legal_unit"
    ]
    print(f"\n🏢 Found {len(legal_records)} legal unit records")

    if legal_records:
        legal_record = legal_records[0]
        payload = legal_record.payload
        print(f"   Company name fields:")
        name_fields = [
            k
            for k in payload.keys()
            if "denomination" in k.lower() or "nom" in k.lower()
        ]
        for field in name_fields:
            value = payload.get(field)
            print(f"     {field}: {value}")

else:
    print("⏭️  No registry records available for debugging")

🔍 Debugging Registry Records (Raw API Data)
📊 Found 33 establishment records

🏭 First Establishment Record:
   External ID: 06150054200040
   Entity Type: establishment
   Registry Updated: 2024-03-30 10:22:44.251000
   Ingested At: 2025-10-23 14:44:13.724458

📋 Raw Payload Keys:
   adresse2Etablissement
   adresseEtablissement
   dateCreationEtablissement
   dateDernierTraitementEtablissement
   etablissementSiege
   nic
   nombrePeriodesEtablissement
   periodesEtablissement
   siren
   siret
   statutDiffusionEtablissement
   trancheEffectifsEtablissement
   uniteLegale

🏷️  Name-related fields found:
   nombrePeriodesEtablissement: 5

📅 Establishment Periods in payload:
   Number of periods: 5
   First period keys: ['dateDebut', 'etatAdministratifEtablissement', 'changementEtatAdministratifEtablissement', 'changementEnseigneEtablissement', 'changementDenominationUsuelleEtablissement', 'activitePrincipaleEtablissement', 'nomenclatureActivitePrincipaleEtablissement', 'changementActiv

## 4. Progress Tracking Analysis

Let's examine how progress tracking works and what information is available during extraction.


In [11]:
# Analyze progress tracking if we have progress updates
if API_TOKEN and "progress_updates" in locals() and progress_updates:
    print("📊 Progress Tracking Analysis")
    print("=" * 40)

    print(f"Total progress updates: {len(progress_updates)}")

    # Show progress phases
    phases = [update.get("phase", "Unknown") for update in progress_updates]
    unique_phases = list(set(phases))
    print(f"Progress phases: {', '.join(unique_phases)}")

    # Show detailed progress information
    print(f"\n📈 Progress Details:")
    for i, update in enumerate(progress_updates):
        print(f"   Update {i + 1}:")
        for key, value in update.items():
            print(f"     {key}: {value}")
        print()

    # Create a simple progress visualization
    print("📊 Progress Visualization:")
    for update in progress_updates:
        phase = update.get("phase", "Unknown")
        processed = update.get("processed_facilities", 0)
        total = update.get("total_facilities", 1)
        percentage = (processed / total * 100) if total > 0 else 0

        # Simple progress bar
        bar_length = 20
        filled_length = int(bar_length * percentage / 100)
        bar = "█" * filled_length + "░" * (bar_length - filled_length)

        print(f"   {phase}: [{bar}] {percentage:.1f}% ({processed}/{total})")

else:
    print("⏭️  No progress updates available for analysis")
    print("   Run the full extraction method above first")

⏭️  No progress updates available for analysis
   Run the full extraction method above first


## 5. Error Handling and Validation

Let's explore how the API client handles errors and validation issues.


In [12]:
# Test error handling with invalid SIREN numbers
if API_TOKEN:
    print("🧪 Testing Error Handling")
    print("=" * 30)

    # Test cases for error handling
    test_cases = [
        ("123", "Too short"),
        ("1234567890", "Too long"),
        ("abcdefghi", "Non-numeric"),
        ("000000000", "Invalid format"),
        ("999999999", "Non-existent"),
    ]

    for invalid_siren, description in test_cases:
        print(f"\n🔍 Testing {description}: {invalid_siren}")
        try:
            # Try company-only extraction
            company_data, facility_count = extract_company_only(
                invalid_siren, client, config
            )
            print(f"   ✅ Unexpected success: {company_data.name}")
        except Exception as e:
            print(f"   ❌ Expected error: {type(e).__name__}: {str(e)[:100]}...")

    # Test validation modes
    print(f"\n🔧 Testing Different Validation Modes:")

    validation_modes = [ValidationMode.STRICT, ValidationMode.LENIENT]

    for mode in validation_modes:
        print(f"\n   Mode: {mode}")
        try:
            test_config = ETLConfig(validation_mode=mode)
            company_data, facility_count = extract_company_only(
                TEST_SIREN, client, test_config
            )
            print(f"   ✅ Success with {mode} validation")
        except Exception as e:
            print(f"   ❌ Error with {mode} validation: {type(e).__name__}")

else:
    print("⏭️  Skipping error handling tests - no API token")

🧪 Testing Error Handling

🔍 Testing Too short: 123
   ❌ Expected error: TypeError: cannot unpack non-iterable coroutine object...

🔍 Testing Too long: 1234567890
   ❌ Expected error: TypeError: cannot unpack non-iterable coroutine object...

🔍 Testing Non-numeric: abcdefghi
   ❌ Expected error: TypeError: cannot unpack non-iterable coroutine object...

🔍 Testing Invalid format: 000000000
   ❌ Expected error: TypeError: cannot unpack non-iterable coroutine object...

🔍 Testing Non-existent: 999999999
   ❌ Expected error: TypeError: cannot unpack non-iterable coroutine object...

🔧 Testing Different Validation Modes:

   Mode: ValidationMode.STRICT
   ❌ Error with ValidationMode.STRICT validation: TypeError

   Mode: ValidationMode.LENIENT
   ❌ Error with ValidationMode.LENIENT validation: TypeError


C:\Users\porti\AppData\Local\Temp\ipykernel_30976\3516957613.py:19: RuntimeWarning: coroutine 'extract_company_only' was never awaited
  company_data, facility_count = extract_company_only(
C:\Users\porti\AppData\Local\Temp\ipykernel_30976\3516957613.py:35: RuntimeWarning: coroutine 'extract_company_only' was never awaited
  company_data, facility_count = extract_company_only(


## 6. Data Export and Serialization

Let's see how to export and serialize the extracted data for further use.


In [13]:
# Data export and serialization examples
if API_TOKEN and "result" in locals():
    print("💾 Data Export and Serialization")
    print("=" * 40)

    # Convert to dictionary format
    def model_to_dict(obj):
        """Convert a model object to dictionary."""
        if hasattr(obj, "__dict__"):
            result = {}
            for key, value in obj.__dict__.items():
                if not key.startswith("_"):
                    if isinstance(value, list):
                        result[key] = [
                            model_to_dict(item) if hasattr(item, "__dict__") else item
                            for item in value
                        ]
                    elif hasattr(value, "__dict__"):
                        result[key] = model_to_dict(value)
                    else:
                        result[key] = value
            return result
        return obj

    # Export company data
    company_dict = model_to_dict(result.company)
    print(f"🏢 Company Data (Dictionary):")
    print(f"   Keys: {list(company_dict.keys())}")
    print(f"   Name: {company_dict.get('name', 'N/A')}")

    # Export facilities data
    facilities_data = []
    for facility in result.facilities[:3]:  # First 3 facilities
        facility_dict = model_to_dict(facility)
        facilities_data.append(facility_dict)

    print(f"\n🏭 Facilities Data (First 3):")
    for i, facility in enumerate(facilities_data):
        print(f"   Facility {i + 1}: {facility.get('name', 'N/A')}")
        print(f"     Keys: {list(facility.keys())}")

    # Export addresses data
    addresses_data = []
    for address in result.addresses[:3]:  # First 3 addresses
        address_dict = model_to_dict(address)
        addresses_data.append(address_dict)

    print(f"\n📍 Addresses Data (First 3):")
    for i, address in enumerate(addresses_data):
        print(f"   Address {i + 1}: {address.get('street', 'N/A')}")
        print(f"     Keys: {list(address.keys())}")

    # Create summary export
    export_data = {
        "extraction_summary": {
            "siren": TEST_SIREN,
            "company_name": result.company.name,
            "total_facilities": len(result.facilities),
            "total_addresses": len(result.addresses),
            "extraction_timestamp": datetime.now().isoformat(),
        },
        "company": company_dict,
        "facilities": facilities_data,
        "addresses": addresses_data,
    }

    print(f"\n📊 Export Summary:")
    print(f"   Total data points: {len(str(export_data))} characters")
    print(f"   Company fields: {len(company_dict)}")
    print(f"   Facility fields: {len(facilities_data[0]) if facilities_data else 0}")
    print(f"   Address fields: {len(addresses_data[0]) if addresses_data else 0}")

else:
    print("⏭️  No extraction results available for export")
    print("   Run the extraction methods above first")

💾 Data Export and Serialization
🏢 Company Data (Dictionary):
   Keys: ['name', 'identifiers', 'creation_date', 'acronym', 'employee_band', 'employee_band_year', 'company_size_category', 'category_year', 'diffusion_status', 'is_purged', 'last_update', 'period_count', 'association_id']
   Name: BDO RHONE-ALPES

🏭 Facilities Data (First 3):
   Facility 1: Unknown Facility
     Keys: ['name', 'identifiers', 'parent_siren', 'nic', 'creation_date', 'is_headquarters', 'employee_band', 'employee_band_year', 'search_score', 'diffusion_status', 'crafts_activity', 'last_update', 'period_count']
   Facility 2: BDO
     Keys: ['name', 'identifiers', 'parent_siren', 'nic', 'creation_date', 'is_headquarters', 'employee_band', 'employee_band_year', 'search_score', 'diffusion_status', 'crafts_activity', 'last_update', 'period_count']
   Facility 3: Unknown Facility
     Keys: ['name', 'identifiers', 'parent_siren', 'nic', 'creation_date', 'is_headquarters', 'employee_band', 'employee_band_year', 'searc

## 7. Practical Usage Examples

Here are some practical examples of how you might use the SIRENE API client in real applications.


In [14]:
# Practical usage examples
print("🚀 Practical Usage Examples")
print("=" * 35)


# Example 1: Batch processing multiple SIRENs
def batch_extract_companies(siren_list, client, config):
    """Extract company data for multiple SIRENs."""
    results = []

    for siren in siren_list:
        try:
            company_data, facility_count = extract_company_only(siren, client, config)
            results.append(
                {
                    "siren": siren,
                    "name": company_data.name,
                    "facility_count": facility_count,
                    "status": "success",
                }
            )
        except Exception as e:
            results.append(
                {
                    "siren": siren,
                    "name": None,
                    "facility_count": 0,
                    "status": "error",
                    "error": str(e),
                }
            )

    return results


# Example 2: Company search by name pattern
def search_companies_by_pattern(pattern, client, config):
    """Search for companies by name pattern (simplified example)."""
    # This is a simplified example - real implementation would use search APIs
    print(f"🔍 Searching for companies matching pattern: '{pattern}'")
    print("   (This would typically use SIRENE search APIs)")
    return []


# Example 3: Data validation and cleaning
def validate_and_clean_data(result):
    """Validate and clean extracted data."""
    cleaned_data = {
        "company": {
            "name": result.company.name.strip() if result.company.name else None,
            "siren": result.company.identifiers[0].value
            if result.company.identifiers
            else None,
        },
        "facilities": [],
        "addresses": [],
    }

    # Clean facilities data
    for facility in result.facilities:
        cleaned_facility = {
            "name": facility.name.strip() if facility.name else None,
            "siret": facility.identifiers[0].value if facility.identifiers else None,
            "is_headquarters": facility.is_headquarters,
        }
        cleaned_data["facilities"].append(cleaned_facility)

    # Clean addresses data
    for address in result.addresses:
        cleaned_address = {
            "street": address.street.strip() if address.street else None,
            "city": address.city.strip() if address.city else None,
            "postal_code": address.postal_code.strip() if address.postal_code else None,
            "coordinates": address.coordinates,
        }
        cleaned_data["addresses"].append(cleaned_address)

    return cleaned_data


# Example 4: Progress monitoring with custom callback
def create_progress_monitor():
    """Create a progress monitor with custom logging."""
    progress_log = []

    def progress_callback(update):
        progress_log.append(
            {
                "timestamp": datetime.now().isoformat(),
                "phase": update.get("phase", "unknown"),
                "processed": update.get("processed_facilities", 0),
                "total": update.get("total_facilities", 0),
                "percentage": (
                    update.get("processed_facilities", 0)
                    / update.get("total_facilities", 1)
                )
                * 100,
            }
        )

        # Custom logging
        print(
            f"📊 [{datetime.now().strftime('%H:%M:%S')}] {update.get('phase', 'Unknown')}: "
            f"{update.get('processed_facilities', 0)}/{update.get('total_facilities', 0)} "
            f"({progress_log[-1]['percentage']:.1f}%)"
        )

    return progress_callback, progress_log


print("✅ Example functions defined:")
print("   - batch_extract_companies(): Process multiple SIRENs")
print("   - search_companies_by_pattern(): Search by name pattern")
print("   - validate_and_clean_data(): Clean and validate data")
print("   - create_progress_monitor(): Custom progress monitoring")

🚀 Practical Usage Examples
✅ Example functions defined:
   - batch_extract_companies(): Process multiple SIRENs
   - search_companies_by_pattern(): Search by name pattern
   - validate_and_clean_data(): Clean and validate data
   - create_progress_monitor(): Custom progress monitoring


## 8. Summary and Next Steps

This notebook has demonstrated the core capabilities of the SIRENE API client. Here's what we've covered:


In [15]:
# Summary of what we've learned
print("📚 Summary of SIRENE API Client Capabilities")
print("=" * 50)

capabilities = [
    "✅ Data Models: Company, Facility, Address, Identifier, Period models",
    "✅ Extraction Methods: Company-only (fast) vs Full extraction (comprehensive)",
    "✅ Progress Tracking: Real-time updates during extraction",
    "✅ Error Handling: Validation modes and exception handling",
    "✅ Data Export: Serialization to dictionaries and JSON",
    "✅ Practical Examples: Batch processing, data cleaning, monitoring",
]

for capability in capabilities:
    print(f"   {capability}")

print(f"\n🎯 Key Takeaways:")
print(f"   1. Use extract_company_only() for quick company information")
print(f"   2. Use extract_and_transform_siren_with_progress() for complete data")
print(f"   3. Progress callbacks provide real-time updates")
print(f"   4. Validation modes control error handling strictness")
print(f"   5. Data models are structured and type-safe")

print(f"\n🚀 Next Steps:")
print(f"   1. Set up your SIRENE_API_TOKEN environment variable")
print(f"   2. Run the extraction examples above with real data")
print(f"   3. Integrate into your application (Django, Flask, etc.)")
print(f"   4. Explore the full API documentation for advanced features")

print(f"\n📖 Documentation:")
print(
    f"   - SIRENE API: https://api.insee.fr/catalogue/site/themes/wso2/subthemes/insee/pages/item-info.jag?name=Sirene&version=V3&provider=insee"
)
print(f"   - sirene-api-client: Check the project repository for full documentation")

📚 Summary of SIRENE API Client Capabilities
   ✅ Data Models: Company, Facility, Address, Identifier, Period models
   ✅ Extraction Methods: Company-only (fast) vs Full extraction (comprehensive)
   ✅ Progress Tracking: Real-time updates during extraction
   ✅ Error Handling: Validation modes and exception handling
   ✅ Data Export: Serialization to dictionaries and JSON
   ✅ Practical Examples: Batch processing, data cleaning, monitoring

🎯 Key Takeaways:
   1. Use extract_company_only() for quick company information
   2. Use extract_and_transform_siren_with_progress() for complete data
   3. Progress callbacks provide real-time updates
   4. Validation modes control error handling strictness
   5. Data models are structured and type-safe

🚀 Next Steps:
   1. Set up your SIRENE_API_TOKEN environment variable
   2. Run the extraction examples above with real data
   3. Integrate into your application (Django, Flask, etc.)
   4. Explore the full API documentation for advanced features
